In [2]:
!pip install -q h5py pyyaml

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.


In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

!pip install -q tensorflow==2.0.0-alpha0
import tensorflow as tf
from tensorflow import keras

tf.__version__

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.


'2.0.0-alpha0'

In [4]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0


11501568/11490434 [==============================] - 3s 0us/step


In [5]:
# Returns a short sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  return model


# Create a basic model instance
model = create_model()
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [6]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model = create_model()

model.fit(train_images, train_labels,  epochs = 10,
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  # pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.


Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 800/1000 [=======================>......] - ETA: 0s - loss: 1.2826 - accuracy: 0.6087
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 1ms/sample - loss: 1.1678 - accuracy: 0.6470 - val_loss: 0.7131 - val_accuracy: 0.7910
Epoch 2/10
 832/1000 [=======================>......] - ETA: 0s - loss: 0.4148 - accuracy: 0.8882
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 501us/sample - loss: 0.4226 - accuracy: 0.8830 - val_loss: 0.5370 - val_accuracy: 0.8380
Epoch 3/10
 928/1000 [==========================>...] - ETA: 0s - loss: 0.2882 - accuracy: 0.9300
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 468us/sample - loss: 0.2885 - accuracy: 0.9300 - val_loss: 0.4763 - val_accuracy: 0.8530
Epoch 4/10
 896/1000 [=========================>....] - ETA: 0s - loss: 0.2049 - accuracy: 0.9475
Epoch 0

In [7]:
!ls {checkpoint_dir}


checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


In [8]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))


1000/1000 [==============================] - 0s 163us/sample - loss: 2.4540 - accuracy: 0.0650
Untrained model, accuracy:  6.50%


In [9]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))


1000/1000 [==============================] - 0s 103us/sample - loss: 0.4189 - accuracy: 0.8690
Restored model, accuracy: 86.90%


In [10]:
# include the epoch in the file name. (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # Save weights, every 5-epochs.
    period=5)

model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)




Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [11]:
! ls {checkpoint_dir}


checkpoint			  cp-0025.ckpt.index
cp-0000.ckpt.data-00000-of-00001  cp-0030.ckpt.data-00000-of-00001
cp-0000.ckpt.index		  cp-0030.ckpt.index
cp-0005.ckpt.data-00000-of-00001  cp-0035.ckpt.data-00000-of-00001
cp-0005.ckpt.index		  cp-0035.ckpt.index
cp-0010.ckpt.data-00000-of-00001  cp-0040.ckpt.data-00000-of-00001
cp-0010.ckpt.index		  cp-0040.ckpt.index
cp-0015.ckpt.data-00000-of-00001  cp-0045.ckpt.data-00000-of-00001
cp-0015.ckpt.index		  cp-0045.ckpt.index
cp-0020.ckpt.data-00000-of-00001  cp-0050.ckpt.data-00000-of-00001
cp-0020.ckpt.index		  cp-0050.ckpt.index
cp-0025.ckpt.data-00000-of-00001


In [12]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

u'training_2/cp-0050.ckpt'

In [13]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))


1000/1000 [==============================] - 0s 164us/sample - loss: 0.4798 - accuracy: 0.8790
Restored model, accuracy: 87.90%


In [14]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Restore the weights
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))


1000/1000 [==============================] - 0s 164us/sample - loss: 0.4798 - accuracy: 0.8790
Restored model, accuracy: 87.90%


In [15]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

# Save entire model to a HDF5 file
model.save('my_model.h5')


Epoch 1/5
1000/1000 [==============================] - 0s 486us/sample - loss: 1.1195 - accuracy: 0.6830
Epoch 2/5
1000/1000 [==============================] - 0s 252us/sample - loss: 0.4075 - accuracy: 0.8900
Epoch 3/5
1000/1000 [==============================] - 0s 266us/sample - loss: 0.2847 - accuracy: 0.9280
Epoch 4/5
1000/1000 [==============================] - 0s 249us/sample - loss: 0.2133 - accuracy: 0.9460
Epoch 5/5
1000/1000 [==============================] - 0s 248us/sample - loss: 0.1593 - accuracy: 0.9640


In [16]:
# Recreate the exact same model, including weights and optimizer.
new_model = keras.models.load_model('my_model.h5')
new_model.summary()


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [17]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))


1000/1000 [==============================] - 0s 176us/sample - loss: 0.4236 - accuracy: 0.8590
Restored model, accuracy: 85.90%


In [18]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)


Epoch 1/5
1000/1000 [==============================] - 0s 353us/sample - loss: 1.0897 - accuracy: 0.7090
Epoch 2/5
1000/1000 [==============================] - 0s 237us/sample - loss: 0.3959 - accuracy: 0.8870
Epoch 3/5
1000/1000 [==============================] - 0s 244us/sample - loss: 0.2694 - accuracy: 0.9310
Epoch 4/5
1000/1000 [==============================] - 0s 239us/sample - loss: 0.1895 - accuracy: 0.9560
Epoch 5/5
1000/1000 [==============================] - 0s 240us/sample - loss: 0.1495 - accuracy: 0.9680


In [19]:
import time
saved_model_path = "./saved_models/{}".format(int(time.time()))

tf.keras.experimental.export_saved_model(model, saved_model_path)
saved_model_path

W0422 11:24:25.489875 139801998976768 deprecation.py:323] From /home/dellie/.local/lib/python2.7/site-packages/tensorflow/python/saved_model/signature_def_utils_impl.py:253: build_tensor_info (from tensorflow.python.saved_model.utils_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
W0422 11:24:25.493488 139801998976768 export_utils.py:182] Export includes no default signature!
W0422 11:24:26.072474 139801998976768 export_utils.py:182] Export includes no default signature!


u'./saved_models/1555928664'

In [20]:
!ls saved_models/


1555928664


In [21]:
new_model = tf.keras.experimental.load_from_saved_model(saved_model_path)
new_model.summary()


Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.predict(test_images).shape

(1000, 10)

In [23]:
# The model has to be compiled before evaluating.
# This step is not required if the saved model is only being deployed.

new_model.compile(optimizer=model.optimizer,  # keep the optimizer that was loaded
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Evaluate the restored model.
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))


1000/1000 [==============================] - 0s 171us/sample - loss: 0.4437 - accuracy: 0.8520
Restored model, accuracy: 85.20%
